In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import astropy
from astropy.io import fits
from astropy.io import ascii

from astropy.table import Table
from astropy.timeseries import TimeSeries
from astropy.time import Time
from astropy.timeseries import TimeSeries, aggregate_downsample
from astropy.coordinates import match_coordinates_sky as coords

from PIL import Image

import warnings
warnings.filterwarnings('ignore')

from mid_IR_variables import fileIO_processing as fp
from mid_IR_variables import lightcurve_processing as lp
from mid_IR_variables import ZTF_lightcurve_processing as zlp
from mid_IR_variables import reverberation_mapper 
from mid_IR_variables import MCMC_sampler


from sklearn.gaussian_process import GaussianProcessRegressor, kernels

from scipy import optimize
import scipy.stats as stats

import os

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 22})
matplotlib.rc('font',**{'family':'serif','serif':['Times']})
matplotlib.rc('text', usetex=True)

SAVE_FILEPATH = 'C:/Users/paiaa/Documents/Research/Blanton Lab/Midir Variables/'
IMPORT_FILEPATH ='C:/Users/paiaa/Documents/Research/Blanton Lab/Midir Variables/'

mnsa_hdu, manga_wise_hdu, pipe3d_hdu = fp.import_manga(6, 1, 1)
mnsa, mwv, pipe3d = mnsa_hdu.data, manga_wise_hdu.data, pipe3d_hdu.data

In [27]:
pifus = np.array([ '10223-3701', '9000-1901', '7977-9101', '9186-9101','8091-6101', '9487-9102', '8593-12705', '8615-3701','7981-9102', '8341-12704', 
                  '8466-3704', '8561-3704', '7815-6104', '8091-9101', '9870-9101', '9514-1901', '9487-3702', '8710-6102', '11833-3702', '11944-12704', 
                  '8553-1901', '12071-12703', '9887-1901', '10516-12701', '7815-6104', '8132-6101', '8612-1901', '8619-12702', '9497-12705', '9502-12703',
                    '12483-9102', '8326-6102', '8714-3704', '8999-3702'])
for pifu in pifus:
    #os.makedirs(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu, exist_ok=True)

    opt_data = lp.generate_combined_lightcurve(pifu)
    size = np.size(opt_data)
    #opt_data.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/optical.dat", format='ascii.no_header', overwrite=True)

    w1 = fp.process_wise(pifu, mwv, band=1)
    w1['time'] = w1['time'].to_value('decimalyear')
    w1_size = np.size(w1)
    #w1.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/w1.dat", format='ascii.no_header', overwrite=True)

    w2 = fp.process_wise(pifu, mwv, band=2)
    w2['time'] = w2['time'].to_value('decimalyear')
    w2_size = np.size(w2)
    #w2.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/w2.dat", format='ascii.no_header', overwrite=True)

using padded times
using padded times
using padded times
using padded times
using padded times
using padded times


In [19]:
def generate_optical_lightcurve(pifu, l=[0.95, 1.05]):

        optical_lightcurve = lp.generate_combined_lightcurve(pifu)

        poly_subtracted_obj_p, fit, fitted_poly = lp.polyfit_lightcurves(optical_lightcurve, deg=10)
        gp, llh, hyperparams, cov = lp.GP(poly_subtracted_obj_p, kernel_num=3, lengthscale=(l[0], l[1]))
        gp_fitted_poly =lp.make_polynomial(gp, fit)
        
        gp['mag']+=gp_fitted_poly
        print('**GP Kernel:', str(hyperparams)+'**')

        return optical_lightcurve, gp

In [64]:
pifu='10223-3701'

opt_data = lp.generate_combined_lightcurve(pifu)
#opt_data['time'] = opt_data['time'].to_value('decimalyear') - opt_data['time'].to_value('decimalyear')[0]

# size = np.size(opt_data)
# a, b, c, d = fp.bin_data(opt_data, 10, 1000)
# b = b[~b['mag'].mask]

# opt_data=Table()
# opt_data['time'] = (b['time_bin_start'].to_value('decimalyear') - b['time_bin_start'][0].to_value('decimalyear'))*365
# opt_data['mag'] = b['mag']
# opt_data['mag_err'] = b['mag_err']
opt_data.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/optical.dat", format='ascii.no_header', overwrite=True)

w1 = fp.process_wise(pifu, mwv, band=1)
w1['time'] = (w1['time'].to_value('decimalyear')-w1['time'].to_value('decimalyear')[0])*365

w1_size = np.size(w1)
w1.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/w1.dat", format='ascii.no_header', overwrite=True)

w2 = fp.process_wise(pifu, mwv, band=2)
w2['time'] = w2['time'].to_value('decimalyear')
w2_size = np.size(w2)
w2.write(SAVE_FILEPATH+"Lightcurves/JAVELIN/"+pifu+"/w2.dat", format='ascii.no_header', overwrite=True)

using padded times


In [65]:
opt_data

time,mag,mag_err
Time,float64,float64
0.0,13.5,0.05
0.009280000633680174,13.51,0.05
0.018380001155264836,13.51,0.05
0.02751000182001917,13.47,0.05
20.961101329390885,13.45,0.05
20.97036133001211,13.46,0.05
20.97960133053789,13.44,0.05
20.98884133114666,13.44,0.05
26.956691709604,13.46,0.05
